## Import Necessary Packages

In [1]:
# Import necessary modules
import os
import time
import math
import pandas as pd
from gtts import gTTS
from playsound import playsound
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Scrape Machine Translation Services

In [8]:
# Define start time
startTime = time.time()
# Define time functions
def convert(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%dh %02dm %02ds" % (hour, minutes, seconds)
def convertTTS(seconds): 
    seconds = seconds % (24 * 3600) 
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
    return "%d hours %02d minutes %02d seconds" % (hour, minutes, seconds)
# Define scraping functions
def scrapeDeepL(language):
    # Define the URL to be opened
    url = 'https://www.deepl.com/translator'
    # Define the driver for Selenium to use
    driver = webdriver.Chrome('chromedriver.exe')
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(2)
    # Open the "Select Language" dropdown menu
    buttonSource = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div/button').click()
    # Sleep to avoid errors
    time.sleep(2)
    if language == '中文':
        # Click the Chinese button
        button = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div/div/button[12]').click()
    else:
        # Click the Japanese button
        button = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[1]/div/div/button[11]').click()
    # Sleeep to avoid errors
    time.sleep(2)
    # Find the text field
    textFieldSource = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[2]/div[1]/textarea')
    # Type the text into the text field
    test = source.loc[0:0].to_string(index = False, columns = None, header = False)
    batch = 1
    sentencesCleanBatches = {}
    sentencesClean = []
    for row in source['origin']:
        textFieldSource.send_keys(row)
        textFieldSource.send_keys(Keys.RETURN)
        try:
            char_usage = driver.find_element_by_xpath('//*[@id="dl_translator"]/div[3]/div[2]/div[1]/div[2]/div[2]').text
            char_usage1 = char_usage[0:4]
            char_usage2 = char_usage1.strip()
            char_count = int(char_usage2)
        except Exception:
            char_count = 0
            pass
        if char_count >= 4500:
            # Pause to avoid errors
            time.sleep(5)
            # Store the translated text as a string
            translatedText = driver.find_element_by_xpath('//*[@id="target-dummydiv"]')
            translatedText_content = translatedText.get_attribute('innerHTML').strip()
            # Sleep to avoid errors
            time.sleep(2)
            # Clean the output text and print
            sentencesCleanBatches[batch] = translatedText_content.split("\n")
            batch += 1
            # Clear the input field
            textFieldSource.clear()
    # Pause to avoid errors
    time.sleep(5)
    # Store the translated text as a string
    translatedText = driver.find_element_by_xpath('//*[@id="target-dummydiv"]')
    translatedText_content = translatedText.get_attribute('innerHTML').strip()
    # Sleep to avoid errors
    time.sleep(2)
    # Clean the output for the last batch
    sentencesCleanBatches[batch] = translatedText_content.split("\n")
    # Concatenate all the batches into once list
    for i in range(1, batch + 1):
        sentencesClean = sentencesClean + sentencesCleanBatches[i]
    # Sleep to avoid errors
    time.sleep(2)
    # Close the browser window
    driver.quit()
    return sentencesClean
def scrapePapago(language):
    # Define the URL to be opened
    url = 'https://papago.naver.com/'
    # Define the driver for Selenium to use
    driver = webdriver.Chrome('chromedriver.exe')
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(2)
    # Find the Text Field and type the sentences
    textFieldSource = driver.find_element_by_xpath('//*[@id="txtSource"]')
    test = source.loc[0:0].to_string(index = False, columns = None, header = False)
    batch = 1
    sentencesCleanBatches = {}
    sentencesClean = []
    for row in source['origin']:
        textFieldSource.send_keys(row)
        textFieldSource.send_keys(Keys.RETURN)
        try:
            char_usage = driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/section/div/div[1]/div[1]/div/p[1]').text
            char_usage1 = char_usage[0:4]
            char_usage2 = char_usage1.strip()
            char_count = int(char_usage2)
        except Exception:
            char_count = 0
            pass
        if char_count >= 4500:
            # Pause to avoid errors
            time.sleep(5)
            # Push the Translate button
            translate_button = driver.find_element_by_xpath('//*[@id="btnTranslate"]').click()
            # Pause to avoid errors
            time.sleep(5)
            # Store the translated text as a string
            translatedText = driver.find_element_by_xpath('//*[@id="txtTarget"]')
            translatedText_content = translatedText.text
            # Clean the output text and print
            sentencesCleanBatches[batch] = translatedText_content.split("\n")
            batch += 1
            # Clear the input field
            textFieldSource.clear()
    # Pause to avoid errors
    time.sleep(5)
    # Push the Translate button
    translate_button = driver.find_element_by_xpath('//*[@id="btnTranslate"]').click()
    # Pause to avoid errors
    time.sleep(5)
    # Store the translated text as a string
    translatedText = driver.find_element_by_xpath('//*[@id="txtTarget"]')
    translatedText_content = translatedText.text
    # Clean the output text and print
    sentencesCleanBatches[batch] = translatedText_content.split("\n")
    # Concatenate all the batches into once list
    for i in range(1, batch + 1):
        sentencesClean = sentencesClean + sentencesCleanBatches[i]
    # Close the browser window
    driver.quit()
    # Return necessary objects
    return sentencesClean
def scrapeGoogleTranslate():
    # Define the URL to be opened
    url = 'https://translate.google.com/'
    # Define the driver for Selenium to use
    driver = webdriver.Chrome('chromedriver.exe')
    # Open the specfied URL
    driver.get(url)
    # Sleep to avoid errors
    time.sleep(2)
    # Text Input Field
    textFieldSource = driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[1]/span/span/div/textarea')
    test = source.loc[0:0].to_string(index = False, columns = None, header = False)
    batch = 1
    sentencesCleanBatches = {}
    sentencesClean = []
    for row in source['origin']:
        textFieldSource.send_keys(row)
        textFieldSource.send_keys(Keys.RETURN)
        try:
            char_usage = driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[1]/div[5]/div[2]/span/span').text
            char_count = int(char_usage)
        except Exception:
            char_count = 0
            pass
        if char_count >= 4500:
            # Pause to avoid errors
            time.sleep(5)
            # Store the translated text as a string
            translatedText = driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]')
            translatedText_content = translatedText.text
            # Clean the output text and print
            sentencesCleanBatches[batch] = translatedText_content.split("\n")
            batch += 1
            # Clear the input field
            textFieldSource.clear()
    # Pause to avoid errors
    time.sleep(5)
    # Store the translated text as a string
    translatedText = driver.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]')
    translatedText_content = translatedText.text
    # Clean the output text and print
    sentencesCleanBatches[batch] = translatedText_content.split("\n")
    # Concatenate all the batches into once list
    for i in range(1, batch + 1):
        sentencesClean = sentencesClean + sentencesCleanBatches[i]
    # Close the driver
    driver.quit()
    return sentencesClean
# Create a DataFrame of the sentences to be translated
source = pd.read_excel('Source.xlsx', header = None)
source.columns = ['origin', 'language', 'extra']
# Get the project name
project = source.language[1]
# Get the language
language = source.language[0]
# Drop the language column
source = source.drop(columns = 'language')
# Drop the extra column
source = source.drop(columns = 'extra')
# Drop nulls
source = source.dropna(axis = 0)
# Determine which scraper to use
if language == '한국어':
    sentencesClean = scrapePapago(language)
    service = 'Papago'
else:
    sentencesClean = scrapeDeepL(language)
    service = 'DeepL'
sentencesCleanGoogle = scrapeGoogleTranslate()
# Create a DataFrame
cols = [language, 'Final', 'Final Count', service, '{} Count'.format(service), '{} Difference'.format(service), 'Google Translate', 'Google Translate Count', 'Google Translate Difference']
df = pd.DataFrame(columns = cols)
df[service] = sentencesClean
df[language] = source['origin']
df['Google Translate'] = sentencesCleanGoogle
# Drop nan rows
df.dropna(how = 'all', inplace = True, axis = 0)
# Export DataFrame as an XLSX to Google Drive\Work\Translation folder
df.to_excel('{}\{}.xlsx'.format(language, project), index=False)
os.startfile('{}\{}.xlsx'.format(language, project))

## Calculate Differences

In [9]:
# (Only necessary when scraping is not done first)
# Variables
language = '한국어'
project = 'difference test'

In [10]:
df1 = pd.read_excel('{}\{}.xlsx'.format(language, project))
df1

,한국어,Final,Papago,Papago Count,Papago Difference,Google Translate,Google Translate Count,Google Translate Difference
0,"포포, 이리 오너라!","Popo, come here!","Popo, come here!",NaN,NaN,"Popo, come here!",NaN,NaN
1,앗! 저 도깨비 녀석!,Oh! That goblin!,Oh! That goblin!,NaN,NaN,Oh! That goblin guy!,NaN,NaN
2,음? 이 아이를 아는가?,Hmm? Do you know this kid?,Hmm? Do you know this kid?,NaN,NaN,Well? Do you know this child?,NaN,NaN
3,저 녀석이 우리를 이곳으로 끌고 왔다고!,He brought us here!,He brought us here!,NaN,NaN,That guy brought us here!,NaN,NaN
4,기껏 도와줬더니!,I helped you as much as I could!,I've helped you as much as I can!,NaN,NaN,I helped you all the way!,NaN,NaN
5,그 전에 그쪽이 먼저 불덩어리를 던지지 않았슙니까?!,Didn't you throw a ball of fire that way before?,Didn't you throw a ball of fire first?!,NaN,NaN,Didn't you throw the fireball first before that?!,NaN,NaN
6,그리고 끌고 간 적 없슙니다!,And he didn't drag her!,And he didn't drag her!,NaN,NaN,And I never dragged it away!,NaN,NaN
7,그쪽이 멋대로 따라온 겁니다!,You've come your own way!,You've come your way!,NaN,NaN,You followed whatever you want!,NaN,NaN
8,따라간 건 아닌데….,I didn't follow you…,I didn't follow you....,NaN,NaN,I didn't follow... .,NaN,NaN
9,그래서 몬스터들한테 두들겨 맞고 있는 걸 구해준 게 누군데?!,So who saved you from being beaten up by the m...,So who saved you from being beaten up by monst...,NaN,NaN,So who saved me from being beaten by monsters?!,NaN,NaN
